# Domain coverage with coordinate-full sensors

Sensor coverage problem solved by means of homology of the Čech complex in a coordinate-full scenario.

## Problem statement

Given a contractible (homotopy equivalent to a point) compact domain $D \subset \mathbb R^d$ we wish to know whether a set of sensors specified as a point cloud $X \subset \mathbb R^d$ with a given coverage radius $R$ cover (contain) $D$. In other words:

$$
\bigcup_{x \in X} B^d(x, R) \stackrel{?}{\supseteq} D
$$

Where $B^d(x, R)$ denotes a d-ball of radius $R$ centered at $x$; the coverage of a sensor at $x$.

### Assumptions

By $\partial D$ we denote the boundary of the domain. We assume that $\partial D$ is covered by some sensors. This helps to clearly define what $D$ is.

## Solution

We construct the Čech complex given the set of sensor coverages as open sets (they are balls). The obtained nerve $N(U_R^{\text{Cech}})$ is homotopy equivalent to the union of all cover regions as per the nerve lemma: the covering is open (balls are open) and arbitrary non-empty intersections of cover regions are contractible. These intersections are contractible because each cover region is a convex set and intersections of convex sets form convex sets which are themselves contractible. 

Since the nerve is homotopy equivalent to the union of all cover regions, we can look for "holes" in the region by inspection of the homology groups. We know that the boundary $\partial D$ is covered and marks the bounds of the domain. Therefore if the region is covered we expect to find no holes, ie the reduced homology is trivial $\dim \tilde H_*(N(U_R^{\text{Cech}})) = 0$.

Important note: this is merely an implication, not a condition for coverage. Homology is not enough to capture the information needed to find contractible spaces (which is what we are after). A popular "non-physical" example is the [Warsaw circle](https://wildtopology.com/bestiary/warsaw-circle) which has trivial reduced homologies but is not contractible.

### Remarks

Solving this using homology groups is not optimal. There exist sophisticated computational geometry ways to solve this problem with less constraints on the domain and sensors. A more powerful idea using relative homology can provide a sufficient condition for coordinate-free sensor networks. This idea is explored in the separate notebook.

## Load sensors

We first load the JSON file describing the sensors network created with the sensors network creator. Feel free to change the loaded file to verify the computation for other cases.

In [2]:
from dataclasses_json import dataclass_json
from dataclasses import dataclass

@dataclass_json
@dataclass(frozen=True)
class Point:
    x: int
    y: int

@dataclass_json
@dataclass(frozen=True)
class SensorsData:
    coverage_radius: int
    sensors: list[Point]

with open('../data/not_covered_sensors.json', 'r') as f:
    data: SensorsData = SensorsData.from_json(f.read())




## Draw the sensors

We visualize the sensors and their coverage.

In [3]:
from ipycanvas import Canvas, hold_canvas

canvas = Canvas(width=700, height=500)


for p in data.sensors:
    canvas.fill_style = "rgba(130, 245, 216, 0.4)"
    canvas.fill_circle(p.x, p.y, data.coverage_radius)
for p in data.sensors:
    canvas.fill_style = "black"
    canvas.fill_circle(p.x, p.y, 3)

canvas

Canvas()

## Construct the Čech complex

`gudhi` library provides no primitive for Čech complexes, but it does provide the Alpha complex which can be interpreted in equivalent manner by change of parameters ($\epsilon_{\text{Cech}}^2 = \alpha_{\text{Alpha}}$)

In [4]:
import gudhi as gd

cech = gd.AlphaComplex(points=[(o.x, o.y) for o in data.sensors], precision='exact')
alpha = data.coverage_radius**2

We finish by checking the betti numbers (dimension of the homology groups) at `alpha` and expect them to be 0 for a covered domain.

In [4]:
cmplx = cech.create_simplex_tree()

cmplx.compute_persistence()

betti = cmplx.persistent_betti_numbers(alpha, alpha)
print(betti)

#                             special case for H_0
print('Is probably covered?', betti[0] == 1 and all(b == 0 for b in betti[1:]))


[1, 4]
Is probably covered? False
